In [1]:
from Acesso import Login
from Query import Query
from Tempo import DataHora
from Moeda import Moeda
from Email import Email
from RemoverArquivo import Remover
from glob import glob
import os
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

querys={

    'Vendas':


    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME,@DIAS SMALLINT

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    IF DAY(@DTBASE)=1

        BEGIN

            SET @DIAS=DAY(@DTBASE)*-1

            SET @DTFIM=DATEADD(DAY,@DIAS,@DTBASE)

            SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

        END;

    ELSE

        BEGIN

            SET @DTFIM=@DTBASE

            SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

        END;


    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Tipo de Operação]<>'OUTROS' AND [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM
    ORDER BY [Data de Faturamento]    
    
    """,

    'Produto':

    """
    
    SELECT * FROM netfeira.vw_produto
    WHERE Fabricante='ARYZTA'
    
    """,

    'Cliente':

    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,

    'Segmento':

    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """,

    'Vendedor':

    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,

    'Supervisor':

    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """

}

In [2]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [3]:
tabelas_df['Cliente']=tabelas_df['Cliente'].merge(tabelas_df['Segmento'],on='ID Segmento',how='inner')[['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento','Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal']]

In [4]:
tabelas_df['Vendas']=tabelas_df['Vendas'].merge(tabelas_df['Produto'],on='SKU',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU','Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG']]

In [5]:
tabelas_df['Vendedor']=tabelas_df['Vendedor'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe','Supervisor']]

In [6]:
tabelas_df['Vendas']=tabelas_df['Vendas'].merge(tabelas_df['Cliente'],on='ID Cliente',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente','CNPJ','Razão Social', 'Nome Fantasia','Matriz','Segmento','Canal','Bairro','Município', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Qtde', 'Unid. VDA',
       'Qtde VDA', 'Valor de VDA', 'Total Geral', 'Total Venda', 'Comissão R$',
       'Margem Bruta R$', 'Cad Vendedor', 'Situação', 'Peso Bruto KG',
       'Peso Líquido KG']]

In [7]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'Supervisor'], dtype='object')

In [8]:
tabelas_df['Vendas']=tabelas_df['Vendas'].merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'CNPJ', 'Razão Social', 'Nome Fantasia', 'Matriz',
       'Segmento', 'Canal', 'Bairro', 'Município', 'ID Vendedor','Vendedor', 'Nome Resumido', 'Equipe', 'Supervisor',
       'Tipo de Pedido', 'Tipo de Operação', 'Tabelas', 'SKU',
       'Cód. Fabricante', 'Produto', 'Status', 'Fabricante', 'Departamento',
       'Seção', 'Categoria', 'Linha', 'Qtde', 'Unid. VDA', 'Qtde VDA',
       'Valor de VDA', 'Total Geral', 'Situação', 'Peso Bruto KG','Peso Líquido KG']]

In [9]:
tabelas_df['Vendas'].columns

Index(['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'CNPJ', 'Razão Social', 'Nome Fantasia', 'Matriz',
       'Segmento', 'Canal', 'Bairro', 'Município', 'ID Vendedor', 'Vendedor',
       'Nome Resumido', 'Equipe', 'Supervisor', 'Tipo de Pedido',
       'Tipo de Operação', 'Tabelas', 'SKU', 'Cód. Fabricante', 'Produto',
       'Status', 'Fabricante', 'Departamento', 'Seção', 'Categoria', 'Linha',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Geral', 'Situação',
       'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [10]:
produto_df=pd.DataFrame()

produto_df=tabelas_df['Vendas'][['SKU', 'Cód. Fabricante', 'Produto','Fabricante','Linha','Qtde','Total Geral','Peso Bruto KG', 'Peso Líquido KG']].groupby(['SKU', 'Cód. Fabricante', 'Produto','Fabricante','Linha'],as_index=False).sum()

produto_df.sort_values('Qtde',ascending=False,inplace=True,ignore_index=True)

produto_df

,SKU,Cód. Fabricante,Produto,Fabricante,Linha,Qtde,Total Geral,Peso Bruto KG,Peso Líquido KG
0,9439,503500226,"PÃO HB BRIOCHE 4"" CT (15X64G)",ARYZTA,PÃO HB ESPECIAL,1502.0,35374.9300,1614.650,1446.426
1,9489,526000033,PÃO DE FORMA AUSTRALIANO PN10 350G,ARYZTA,ARTESANAL,1449.0,12769.4100,521.640,507.150
2,9445,525200091,BOLO DE LARANJA PN10 - 330G,ARYZTA,BOLO PN10,1432.0,15721.0400,572.800,429.600
3,9491,503500149,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,ARYZTA,FLOWPACK,1418.0,6754.6900,306.288,283.600
4,9440,503500245,PÃO HB BRIOCHE MAX CT (12X96G),ARYZTA,PÃO HB ESPECIAL,1220.0,25770.8600,1244.400,1037.000
5,9446,525200090,BOLO DE CHOCOLATE PN10 - 330G,ARYZTA,BOLO PN10,1158.0,13305.8900,463.200,347.400
6,9461,503500115,PÃO HB MONACO CT C/ GERGELIM CXG (12X75G),ARYZTA,PÃO HB MONACO,936.0,14695.5800,936.000,842.400
7,9417,526300016,PETIT GATEAU CHOCOLATE 2UND DE 85G - 170G PCT ...,ARYZTA,SOBREMESA,676.0,6113.1700,121.680,114.920
8,9492,503500308,PÃO HB PN10 AUSTRALIANO FP (PCT 4UNID X 64G),ARYZTA,FLOWPACK,531.0,4058.5400,144.432,136.467
9,9490,503500148,PÃO HB TRADICIONAL PN10 FP ARYZTA (PCT 4UNID ...,ARYZTA,FLOWPACK,444.0,2029.0700,124.320,114.108


In [11]:
linha_df=pd.DataFrame()

linha_df=produto_df[['Linha','Qtde','Total Geral','Peso Bruto KG', 'Peso Líquido KG']].groupby(['Linha'],as_index=False).sum()

linha_df.sort_values('Qtde',ascending=False,inplace=True,ignore_index=True)

linha_df

,Linha,Qtde,Total Geral,Peso Bruto KG,Peso Líquido KG
0,BOLO PN10,2972.0,33636.9000,1188.800,891.600
1,PÃO HB ESPECIAL,2858.0,66031.2000,3970.810,3490.626
2,FLOWPACK,2837.0,16519.8000,695.808,648.283
3,ARTESANAL,1949.0,27599.0400,1325.976,1241.550
4,PÃO HB MONACO,1034.0,16031.2200,1034.000,930.600
5,SOBREMESA,676.0,6113.1700,121.680,114.920
6,RIVIERA,300.0,3835.0000,251.700,225.000
7,PÃO HB RIVIERA,223.0,3180.1100,178.400,167.250
8,BOLO CANDY,209.0,3860.6500,132.297,125.400
9,PÃES DE HOT DOG,109.0,1585.2025,94.067,81.750


In [12]:
tabelas_df['Vendas']['Data de Faturamento'].max()

Timestamp('2022-08-18 00:00:00')

In [13]:
tabelas_df['Vendas']['Data de Faturamento'].min()

Timestamp('2022-08-01 00:00:00')